## Oracle AI Data Platform v1.0

Copyright © 2025, Oracle and/or its affiliates.

Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

# Execute Oracle SQL on Oracle ALH

## Prerequisites
1. install oracledb in your cluster (create requirements.txt and add oracledb package)
2. upload your tnsnames.ora and ewallet.pem from your wallet into the workspace

## Overview

This example defines a function you can reuse, it generates a sample SQL query and executes it.

In [ ]:
# Define the Oracle function execute_oracle_sql to execute simple SQL statements on Oracle ALH
#
# Parameters:
#  - v_sql_file_path: path to Oracle SQL file in workspace or volume
#  - v_config_dir: directry where your tnsnames.ora resides
#  - v_wallet_dir: directory where your ewallet.pem resides
#  - v_user: Oracle database user
#  - v_password: Oracle database user password
#  - v_dsn: Oracle DSN
#  - v_wallet_password: password fow wallet
#  - fail_on_error: True/False if statement fails, setting to True will stop the script execution, otherwise it till continue
#
import oracledb
 
def execute_oracle_sql(v_sql_file_path, v_config_dir, v_wallet_dir, v_user, v_password, v_dsn, v_wallet_password, fail_on_error=False):
  # Read the SQL file
  sql_script=""
  with open(v_sql_file_path, 'r') as file:
    sql_script = file.read()
 
  # Split the script into individual statements (naive split by semicolon)
  statements = [stmt.strip() for stmt in sql_script.split(';') if stmt.strip()]
  try:
    # Connect to Oracle
    with oracledb.connect(
     config_dir=v_config_dir,
     user=v_user,
     password=v_password,
     dsn=v_dsn,
     wallet_location=v_wallet_dir,
     wallet_password=v_wallet_password) as connection:
        with connection.cursor() as cursor:
            for statement in statements:
                print(f"Executing: {statement}")
                try:
                  cursor.execute(statement)
                except Exception as e:
                  if (fail_on_error):
                      raise e;
                  else:
                      print("  Statement failed (but continuing):", e)
        connection.commit()
        print("SQL script executed successfully.")
  except Exception as e:
    print("Error:", e)


## Generate Sample SQL script
End each statement with a ; it can be multi-line, no PLSQL blocks supported as simplified parsing used. It can also be DDL to create tables.

In [ ]:
%%writefile /Workspace/my_oracle_script.sql
select "Connected" msg from dual;
select "Connected" msg from dual;

## Execute Sample SQL Script

Update the variables below with your values.

In [ ]:
sql_file_path = "/Workspace/my_oracle_script.sql"
config_dir="/Workspace/your_folder_location_for_tns_names_ora",
user="your_user",
password="",
dsn="your_tns_alias",
wallet_location="/Workspace/your_folder_location_for_wallet_pem",
wallet_password=""
 
execute_oracle_sql(sql_file_path, config_dir, wallet_location, user, password, dsn, wallet_password)
